In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image
from numpy.core._multiarray_umath import ndarray
from scipy.integrate import odeint
import copy

Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


def fanin_init(size, fanin=None):
    """
    weight initializer known from https://arxiv.org/abs/1502.01852
    :param size:
    :param fanin:
    :return:
    """
    fanin = fanin or size[0]
    v = 1. / np.sqrt(fanin)
    return torch.Tensor(size).uniform_(-v, v)


class ActorNet(nn.Module):

    def __init__(self):
        super(ActorNet, self).__init__()
        self.fc1 = nn.Linear(2, 10)
        self.fc1.weight.data = fanin_init(self.fc1.weight.data.size())
        self.fc2 = nn.Linear(10, 50)
        self.fc2.weight.data = fanin_init(self.fc2.weight.data.size())
        self.fc3 = nn.Linear(50, 1)
        self.fc3.weight.data.uniform_(-0.03, 0.03)

    def forward(self, x):
        out1 = F.relu(self.fc1(x))
        out2 = F.relu(self.fc2(out1))
        out3 = 20*F.tanh(self.fc3(out2))
        return out3


class CriticNet(nn.Module):

    def __init__(self):
        super(CriticNet, self).__init__()
        self.fc1 = nn.Linear(3, 25)
        self.fc1.weight.data = fanin_init(self.fc1.weight.data.size())
        self.fc2 = nn.Linear(25, 100)
        self.fc2.weight.data = fanin_init(self.fc2.weight.data.size())
        self.fc3 = nn.Linear(100, 1)
        self.fc3.weight.data.uniform_(-0.03, 0.03)

    def forward(self, x):
        out1 = F.sigmoid(self.fc1(x))
        out2 = F.sigmoid(self.fc2(out1))
        out3 = self.fc3(out2)
        return out3


class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.position = 0
        self.memory = []

    def __len__(self):
        return len(self.memory)

    def push(self, *args):
        if len(self) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = self.position + 1

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)


def func_inverted(x, t, u):
    mp = 2
    mc = 10
    a = 1 / (mp + mc)
    g = 9.8
    s = 0.5
    x1, x2 = x
    f1 = (g - a * mp * s * np.power(x2, 2)) / (4 * s / 3 - a * mp * s * np.power(np.cos(x1), 2)) * (np.sin(x1) / x1)
    f2 = -a * np.cos(x1) / (4 * s / 3 - a * mp * s * np.power(np.cos(x1), 2))
    A = np.array([[0, 1], [f1, 0]])
    B = np.array([[0], [f2]])
    # G = np.array([1800, 600]).reshape(1,2)
    # u = np.matmul(G, x)
    dx: float = np.matmul(A, x) + np.matmul(B, u)
    return dx


def func_spring(x, t, u):
    k = 6
    m = 1
    a = 0.3
    c = 2
    x1, x2 = x
    A = np.array([[0, 1], [(-k / m) * (1 + np.power(a, 2) * np.power(x1, 2)), -c / m]])
    B = np.array([[0], [1 / m]])
    dx: float = np.matmul(A, x) + np.matmul(B, u)
    return dx


x_init = np.array([0.5, 2])
t = [0, 0.05]
u = np.array([0.05])
y = odeint(func_spring, x_init, t, args=(u,))

BATCH_SIZE = 128
GAMMA = 0.99
EpisodeNum = 500
TimeStep = 500
REWARD_THRESH = 0.1
Replay_Times = 1
UPDATE_TH = 5

actor = ActorNet()
critic = CriticNet()
actor_target = copy.deepcopy(actor)
critic_target = copy.deepcopy(critic)
memory = ReplayMemory(2000000)
optimizer_actor = optim.Adam(actor.parameters(), lr=0.0001)
optimizer_critic = optim.Adam(critic.parameters(), lr=0.001)

steps_in_episode = []
return_in_episode = []
update_T = 0
for i in range(EpisodeNum):
    print(i)
    s1 = 0.8 * float(np.random.uniform(-3, 3, 1))
    # s2 = float(np.random.uniform(-4, 4, 1))
    s2 = 0
    s_current = torch.tensor([s1, s2], dtype=torch.float32)
    current_return = 0

    for time_step in range(TimeStep):
        explore_std = min(50 / (i + 1), 0.5)
        # control_input = actor.forward(s_current)
        control_input = torch.empty(1).normal_(mean=float(actor.forward(s_current).detach()),
                                               std=explore_std * np.abs(float(actor.forward(s_current).detach())))
        state = s_current
        action = control_input

        s_next_num = odeint(func_spring, s_current.numpy(), [0, 0.05], args=(control_input.detach().numpy(),))[-1, :]
        s_next = torch.tensor(s_next_num, dtype=torch.float32)
        next_state = s_next

        s_next_num_1, s_next_num_2 = s_next_num

        if np.absolute(s_next_num_1) > 3 or np.absolute(s_next_num_2) > 10:
            steps_in_episode = np.append(steps_in_episode, time_step)
            return_in_episode = np.append(return_in_episode, current_return)
            break

        if np.absolute(s_next_num_1) < REWARD_THRESH:
            reward = torch.tensor(1, dtype=torch.float32)
        else:
            reward = torch.tensor(0, dtype=torch.float32)

        current_return = current_return + reward

        memory.push(state, action, next_state, reward)

        if len(memory) >= BATCH_SIZE:
            for rep in range(Replay_Times):
                mini_batch_experience = memory.sample(BATCH_SIZE)
                mini_batch_experience_unzipped = Transition(*zip(*mini_batch_experience))
                mini_batch_state = torch.cat(mini_batch_experience_unzipped.state).reshape(BATCH_SIZE, 2)
                mini_batch_next_state = torch.cat(mini_batch_experience_unzipped.next_state).reshape(BATCH_SIZE, 2)
                mini_batch_action = torch.cat(mini_batch_experience_unzipped.action).clone().detach().reshape(
                    BATCH_SIZE, 1) / 1
                mini_batch_reward = torch.tensor(mini_batch_experience_unzipped.reward)

                Q_value_input = torch.cat([mini_batch_state, mini_batch_action], 1)
                Q_value_critic = critic.forward(Q_value_input)
                with torch.no_grad():
                    action_next_state = actor_target.forward(mini_batch_next_state) / 1
                    Q_value_input_next = torch.cat([mini_batch_next_state, action_next_state], 1)
                    Q_value_target = GAMMA * critic_target(Q_value_input_next) + mini_batch_reward.reshape(BATCH_SIZE, 1)

                critic.zero_grad()
                loss_critic = F.smooth_l1_loss(Q_value_critic, Q_value_target.detach())
                # loss = nn.MSELoss()
                # loss_critic = loss(Q_value_critic, Q_value_target.detach())
                loss_critic.backward()
                # for param in critic.parameters():
                # param.grad.data.clamp_(-10, 10)
                # param.grad.data
                optimizer_critic.step()

                actor.zero_grad()
                actor_loss = - critic.forward(torch.cat([mini_batch_state, actor.forward(mini_batch_state)], 1))
                actor_loss = actor_loss.mean()
                actor_loss.backward()
                # for param in actor.parameters():
                # param.grad.data.clamp_(-1000, 1000)
                # param.grad.data
                optimizer_actor.step()

        s_current = s_next
        update_T = update_T + 1
        if update_T % UPDATE_TH == 0:
            actor_target = copy.deepcopy(actor)
            critic_target = copy.deepcopy(critic)

        if time_step == TimeStep - 1:
            steps_in_episode = np.append(steps_in_episode, time_step)
            return_in_episode = np.append(return_in_episode, current_return)


def func_spring_simu(x, t):
    k = 6
    m = 1
    a = 0.3
    c = 2
    x1, x2 = x
    state = torch.tensor([x1, x2], dtype=torch.float32)
    A = np.array([[0, 1], [(-k / m) * (1 + np.power(a, 2) * np.power(x1, 2)), -c / m]])
    B = np.array([[0], [1 / m]])
    u = actor.forward(state).detach().numpy()
    # u = np.array([0])
    dx: float = np.matmul(A, x) + np.matmul(B, u)
    return dx


def func_inverted_simu(x, t):
    mp = 2
    mc = 10
    a = 1 / (mp + mc)
    g = 9.8
    s = 0.5
    x1, x2 = x
    f1 = (g - a * mp * s * np.power(x2, 2)) / (4 * s / 3 - a * mp * s * np.power(np.cos(x1), 2)) * (np.sin(x1) / x1)
    f2 = -a * np.cos(x1) / (4 * s / 3 - a * mp * s * np.power(np.cos(x1), 2))
    A = np.array([[0, 1], [f1, 0]])
    B = np.array([[0], [f2]])
    u = actor.forward(state).detach().numpy()
    dx: float = np.matmul(A, x) + np.matmul(B, u)
    return dx


x_init = np.array([3, -2])
t = np.linspace(0, 5, 100)
y = odeint(func_spring_simu, x_init, t)

u_simu = actor.forward(torch.tensor(y, dtype=torch.float32)).detach().numpy()

plt.figure()
plt.plot(t, y[:, 0], 'b')
plt.plot(t, y[:, 1], 'r')
plt.figure()
plt.step(t, u_simu)

plt.figure()
plt.plot(steps_in_episode)
plt.xlabel('Episode')
plt.ylabel('Steps Last')

plt.figure()
plt.plot(return_in_episode)
plt.xlabel('Episode')
plt.ylabel('Return Episode')
